In [ ]:
import os
import csv
from datetime import date, datetime
import yaml
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from experiment.toolkits.plotting_helper import *

project_dir = Path().resolve().parents[1]
sys.path.insert(0, str(project_dir))
from time import sleep
import pickle
import datetime
from scipy.stats import linregress
import matplotlib.ticker as ticker

In [ ]:
os.chdir(str(project_dir))

In [ ]:
base_cmd = str(project_dir) + "/bin/modules/image-processing-benchmarking/filtering-test"
def get_processing_time (N_t, kernel_size, reps):
    psf_bin_path = str(project_dir) + "/resources/psf/psf_" + str(N_t) + '_' + str(kernel_size) + ".bin"
    if not os.path.isfile(psf_bin_path):
        psf_pickle_path = str(project_dir) + "/resources/psf/psf_" + str(N_t) + '_' + str(kernel_size) + ".pickle"
        os.popen("python " + str(project_dir) + "/tools/generate-fake-psf.py" + " " + str(N_t) + " " + str(kernel_size) + " " + psf_pickle_path).read()
        os.popen("python " + str(project_dir) + "/tools/psf-translator.py --pickle " + psf_pickle_path + " --address " + psf_bin_path).read()
    command = base_cmd + " " + psf_bin_path + " " + str(N_t) + ' ' + str(kernel_size) + ' ' + str(reps) 
    return os.popen(command).read()

In [ ]:
reps = 1000
N_t = [8,16,32,64,128,256,512,1024,2048] 
kernel_size = [1, 3, 5, 7, 9]
results = [] 

for i in range(len(kernel_size)):
    print(f"{i}/{len(kernel_size)}")

    ks = kernel_size[i]
    temp = []
    for nt in N_t:
        acq_time_str = get_processing_time(nt, ks, reps)
        temp.append((float(acq_time_str.split('\n')[0]), float(acq_time_str.split('\n')[1])))
    results.append(temp)

# Save results

In [ ]:
# Saving the results 
file_name = datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(str(project_dir), "resources", "image-processing-characterization", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": N_t,
            "kernels": kernel_size,
            "experiment": file_name,
            "data": results 
        }, file
        )

# Load results

In [ ]:
load_latest = False 
if not load_latest:
    file_name = "2024-06-13, 10-54-01.pickle"

with open(os.path.join(str(project_dir), "resources", "image-processing-characterization", file_name), "rb") as file:
    data = pickle.load(file)
    N_t = data["xvals"]
    kernel_size = data["kernels"]
    title = data["experiment"]
    results = data["data"]

In [ ]:
import experiment.toolkits.figure_formatting.formatter_2023_az as fmt
import experiment.toolkits.figure_formatting.production_quality_plots as prod_plot

In [ ]:
def xticks(ax, lst):
    ax.set_xticks(lst)
    diff = (lst[-1] - lst[0]) * 0.05
    ax.set_xlim(lst[0] - diff, lst[-1] + diff)
def yticks(ax, lst):
    ax.set_yticks(lst)
    diff = (lst[-1] - lst[0]) * 0.05
    ax.set_ylim(lst[0] - diff, lst[-1] + diff)

In [ ]:
xvals = N_t 
kernel_1 = results[0]
kernel_3 = results[1]
kernel_5 = results[2]
kernel_7 = results[3]
result_1 = linregress(xvals, [x[0] * 1e3 for x in kernel_1])
result_3 = linregress(xvals, [x[0] * 1e3 for x in kernel_3])
result_5 = linregress(xvals, [x[0] * 1e3 for x in kernel_5])
result_7 = linregress(xvals, [x[0] * 1e3 for x in kernel_7])
print(result_1.intercept, result_1.slope)
print(result_3.intercept, result_3.slope)
print(result_5.intercept, result_5.slope)
print(result_7.intercept, result_7.slope)
fig, ax = plt.subplots() 
ax.set_xlabel("Number of traps")
ax.set_ylabel("Runtime (ms)")
PlotPlotter(
    fig,
    ax,
    xvals,
    [x * result_1.slope + result_1.intercept for x in xvals],
    style = {"linestyle" : "-", "color": "#648EF7", "lw": 1}
).draw()
PlotPlotter(
    fig,
    ax,
    xvals,
    [x * result_3.slope + result_3.intercept for x in xvals],
    style = {"linestyle" : "-", "color": "#E0BB00", "lw": 1}
).draw()
PlotPlotter(
    fig,
    ax,
    xvals,
    [x * result_5.slope + result_5.intercept for x in xvals],
    style = {"linestyle" : "-", "color": "#57B52C", "lw": 1}
).draw()
PlotPlotter(
    fig,
    ax,
    xvals,
    [x * result_7.slope + result_7.intercept for x in xvals],
    style = {"linestyle" : "-", "color": "#C46BC7", "lw": 1}
).draw()
prod_plot.production_quality_scatter(fig, ax, [xvals, xvals, xvals, xvals], [[(x[0] * 1e3) for x in kernel_1], [(x[0] * 1e3) for x in kernel_3], [(x[0] * 1e3) for x in kernel_5], [(x[0] * 1e3) for x in kernel_7]], yerr_list=[[(x[1] * 1e3) for x in kernel_1], [(x[1] * 1e3) for x in kernel_3], [(x[1] * 1e3) for x in kernel_5], [(x[1] * 1e3) for x in kernel_7]], save_path=str(project_dir) + "/image-processing.pdf")
ax.legend(["1x1", "3x3", "5x5", "7x7"],fontsize=4)
yticks(ax, [0, 1, 2, 3, 4])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
xticks(ax, [0, 512, 1024, 1536, 2048])
plt.savefig("image-processing.pdf", format='pdf')
plt.show()